In [ ]:
import pandas as pd
import numpy as np
from io import StringIO

Files Needed: <br>
* Schools.csv <br>
* Student_Course_SQL.csv <br>

In [ ]:
NVSchools = pd.read_csv("Schools.csv")

In [ ]:
NVStudents = pd.read_csv("Student_Course_SQL.csv")

# Questions:
What do we do about junior high kids w/out homeroom? AZ and NV<br>

In [ ]:
NVStudents.info()

In [ ]:
NVStudents['grade'].unique()

In [ ]:
# Need to coerce grade to 0-8

In [ ]:
NVStudents

In [ ]:
NVStudents.grade.value_counts()

In [ ]:
NVStudents['Int_Grade'] = np.where(NVStudents['grade'] == '0K', 0, NVStudents['grade'])

In [ ]:
NVStudents.Int_Grade = pd.to_numeric(NVStudents.Int_Grade, errors="coerce")

In [ ]:
NVStudents.Int_Grade.value_counts()

In [ ]:
NVJuniorHigh = NVStudents.loc[NVStudents['Int_Grade'] > 6]

In [ ]:
NVJuniorHigh.grade.value_counts()

In [ ]:
NVJuniorHigh.grade = pd.to_numeric(NVJuniorHigh.grade, errors="coerce")

In [ ]:
NVJuniorHigh.grade.value_counts()

In [ ]:
NV_JR_Homeroom = NVJuniorHigh.loc[NVJuniorHigh['periodstart'] == '7'] 

In [ ]:
NV_JR_Homeroom.info()

In [ ]:
NV_JR_Homeroom

In [ ]:
NV_JR_Homeroom.periodstart = pd.to_numeric(NV_JR_Homeroom.periodstart, errors="coerce")

In [ ]:
NV_JR_Homeroom.grade.value_counts()

In [ ]:
NV_Elem_Homeroom = NVStudents[NVStudents['coursename'] == 'Homeroom']

In [ ]:
NV_Elem_Homeroom.info()

In [ ]:
NV_Elem_Homeroom.Int_Grade.value_counts()

In [ ]:
NV_Elem_Homeroom.info()

In [ ]:
AllNVHomerooms = NV_Elem_Homeroom.append(NV_JR_Homeroom)

In [ ]:
AllNVHomerooms.info()

In [ ]:
transform_grade = {
    39438:'North Valley',
    39439: 'Cadence',
    39440: 'Southwest Las Vegas',
}

AllNVHomerooms['School_Bldg'] = AllNVHomerooms['number'].map(transform_grade)

In [ ]:
AllNVHomerooms['Unique_StudentID'] = 'LTSNV' + AllNVHomerooms['studentNumber'].astype(str)

In [ ]:
AllNVHomerooms['DOB'] = pd.to_datetime(AllNVHomerooms['birthdate'])
AllNVHomerooms['DOB_MDY'] = AllNVHomerooms['DOB'].dt.strftime('%m/%d/%Y')

In [ ]:
AllNVHomerooms['District'] = 'Legacy Traditional Schools- NV'

<H5>  Classroom based on Homeroom and Code needed

In [ ]:
AllNVHomerooms.info()

In [ ]:
AllNVHomerooms['HomeRoom_Last'] = AllNVHomerooms['teacherfullname'].str.partition(', ')[0]

In [ ]:
AllNVHomerooms['Class'] =  AllNVHomerooms['Int_Grade'].astype(str) + ' - ' + AllNVHomerooms['HomeRoom_Last']

In [ ]:
AllNVHomerooms.info()

In [ ]:
dict = {'District' : 'District/Area',
        'School_Bldg' : 'School/Building',
        'number' : 'School/Building Code',
        'Class' : 'Class',
        'firstname' : 'First Name',
        'lastname' : 'Last Name',
        'middlename' : 'Middle Name',
        'Unique_StudentID' : 'Unique Student ID',
        'DOB_MDY' : 'Date of Birth',
        'gender' : 'Gender',
        'Int_Grade' : 'Grade'
       }
AllNVHomerooms.rename(columns=dict, inplace=True)

In [ ]:
AllNVHomerooms['Code'] = '' 

In [ ]:
df_NV_Roster = AllNVHomerooms.filter(['District/Area', 'School/Building', 'School/Building Code', 'Class', 'Code', 'First Name', 'Last Name', 'Middle Name', 
                                        'Unique Student ID', 'Date of Birth', 'Gender', 'Grade'])

In [ ]:
df_NV_Roster.info()

In [ ]:
df_NV_Roster.to_csv('NV_CoGAT_student.csv', index=False, encoding='utf-8')

<H4> Fields for locations file <br>
   * District/Area <br>
   * School/Building <br>
   * School/Building Code <br>
   * Class <br>
   * Grade <br>
   * Code <br>
   * Address <br>
   * City <br>
   * State <br> 
   * Zip Code <br>

In [ ]:
# Just need Address/City/State/Zip Code

In [ ]:
df_NV_Locations = df_NV_Roster[['District/Area', 'School/Building', 'School/Building Code', 'Class', 'Grade']]

In [ ]:
df_NV_Locations

In [ ]:
df_NV_Loc_dd = df_NV_Locations.drop_duplicates()

In [ ]:
df_NV_Loc_dd

In [ ]:
# Get schools
NVSchools

In [ ]:
df_MergeLocs= df_NV_Loc_dd.merge(NVSchools, left_on='School/Building Code',right_on='number')

In [ ]:
df_MergeLocs

In [ ]:
df_NV_Schools_Fields = df_MergeLocs.filter(['District/Area', 'School/Building', 'School/Building Code', 'Class', 'Grade', 
                                            'address', 'city', 'state', 'zip'])

In [ ]:
df_NV_Schools_Fields

In [ ]:
dict = {'address' : 'Address',
        'city' : 'City',
        'state' : 'State',
        'zip' : 'Zip Code'
       }
df_NV_Schools_Fields.rename(columns=dict, inplace=True)

In [ ]:
df_NV_Schools_Fields

In [ ]:
df_NV_Schools_Fields.to_csv('NV_CoGAT_locations.csv', index=False, encoding='utf-8')